In [5]:
import pandas as pd

res = pd.read_csv("/Users/lemanguluzada/Desktop/cognitive-distortion-llm/results/llm_results.csv")   # LLM nəticələri
df  = pd.read_csv("/Users/lemanguluzada/Desktop/cognitive-distortion-llm/data/merged.csv")           # gold label-lı dataset

len(res), res["index"].min(), res["index"].max(), df.shape


(3751, 0, 3750, (8676, 3))

In [6]:
# Gold label-lı dataframe-ə index sütunu əlavə edirik
gold = df.reset_index().rename(columns={"index": "index", "Label": "gold_label"})

merged_res = res.merge(gold[["index", "gold_label"]], on="index", how="left")
merged_res.head()


,index,text,extracted,reasoning,predicted_label,gold_label
0,0,The voice are always fimilar (someone she know...,"""The voice are always familiar (someone she kn...",The statement reflects a worry that the famili...,Emotional Reasoning,Personalization
1,1,I feel trapped inside my disgusting self and l...,"""I feel trapped inside my disgusting self"", ""I...",The person is interpreting their feelings of s...,Labeling,Labeling
2,2,I refused to go because I didn’t know if it wa...,I didn’t know if it was going to go on my reco...,The statement shows concern about an uncertain...,Fortune-telling,Fortune-telling
3,3,I felt like the response was totally irrationa...,"the response was totally irrational, mean and ...",The statement assumes that the response was ir...,Emotional Reasoning,Magnification
4,4,"I thought that he displayed traits of honor, l...","traits of honor, loyalty, integrity, honesty, ...",The statement attributes a wide range of posit...,Labeling,Labeling


In [7]:
merged_res["predicted_label"].value_counts()


predicted_label
Fortune-telling            642
Mind Reading               613
Overgeneralization         606
Labeling                   435
Emotional Reasoning        400
All-or-Nothing Thinking    309
No Distortion              222
Should statements          182
Personalization            174
ERROR                       65
Magnification               54
Mental filter               49
Name: count, dtype: int64

In [8]:
error_mask = merged_res["predicted_label"].isin(["ERROR", "PARSE_ERROR", "RATE_LIMIT_ERROR"])
error_count = error_mask.sum()
total = len(merged_res)
error_count, total, error_count / total


(65, 3751, 0.0173287123433751)

In [9]:
valid = merged_res[~merged_res["predicted_label"].isin(["ERROR", "PARSE_ERROR", "RATE_LIMIT_ERROR"])]
valid = valid.dropna(subset=["gold_label"])

len(valid), valid.head()


(3686,
    index                                               text  \
 0      0  The voice are always fimilar (someone she know...   
 1      1  I feel trapped inside my disgusting self and l...   
 2      2  I refused to go because I didn’t know if it wa...   
 3      3  I felt like the response was totally irrationa...   
 4      4  I thought that he displayed traits of honor, l...   
 
                                            extracted  \
 0  "The voice are always familiar (someone she kn...   
 1  "I feel trapped inside my disgusting self", "I...   
 2  I didn’t know if it was going to go on my reco...   
 3  the response was totally irrational, mean and ...   
 4  traits of honor, loyalty, integrity, honesty, ...   
 
                                            reasoning      predicted_label  \
 0  The statement reflects a worry that the famili...  Emotional Reasoning   
 1  The person is interpreting their feelings of s...             Labeling   
 2  The statement shows conce

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = valid["gold_label"]
y_pred = valid["predicted_label"]

print(classification_report(y_true, y_pred))

cm = confusion_matrix(y_true, y_pred, labels=sorted(y_true.unique()))
cm


                         precision    recall  f1-score   support

All-or-Nothing Thinking       0.32      0.34      0.33       289
    Emotional Reasoning       0.23      0.26      0.24       343
        Fortune-telling       0.30      0.58      0.39       328
               Labeling       0.51      0.62      0.56       362
          Magnification       0.20      0.06      0.09       194
          Mental filter       0.24      0.10      0.14       119
           Mind Reading       0.60      0.69      0.64       536
          No Distortion       0.68      0.26      0.38       574
     Overgeneralization       0.38      0.46      0.42       496
        Personalization       0.68      0.35      0.46       341
      Should statements       0.21      0.37      0.27       104

               accuracy                           0.42      3686
              macro avg       0.40      0.37      0.36      3686
           weighted avg       0.46      0.42      0.41      3686



array([[ 98,  30,  19,  37,   5,   5,  12,  10,  63,   6,   4],
       [ 31,  90,  79,  19,   8,   1,  27,   8,  58,   9,  13],
       [ 21,  26, 190,  12,   1,   0,  47,   0,  26,   0,   5],
       [ 22,  20,   9, 223,   1,   1,  21,   9,  47,   6,   3],
       [ 17,  60,  23,  10,  11,   8,  20,   9,  30,   5,   1],
       [  8,  23,  20,   4,   4,  12,   3,  13,  30,   1,   1],
       [  8,   8,  80,  16,   2,   1, 370,   2,  35,   8,   6],
       [ 34,  54, 105,  22,   9,  10,  55, 152,  38,  12,  83],
       [ 51,  37,  88,  38,   3,   8,  22,   4, 230,   6,   9],
       [ 12,  37,  24,  49,   5,   1,  34,   4,  37, 119,  19],
       [  7,  15,   5,   5,   5,   2,   2,  11,  12,   2,  38]])

In [15]:
labels = sorted(y_true.unique())
print("Labels:", labels)


Labels: ['All-or-Nothing Thinking', 'Emotional Reasoning', 'Fortune-telling', 'Labeling', 'Magnification', 'Mental filter', 'Mind Reading', 'No Distortion', 'Overgeneralization', 'Personalization', 'Should statements']
